Para poder Ejecutar todo el scrapy necesitamos instalar  Selenium
Que la libreria que usamos para poder interactuar con la pagina web para poder pasar los dias para obterner los precios
para installar necesitamos usar este comando en nuestro entorno de desarrollo
pip install selenium

Aqui usamos pandas para poder imprimir en pantalla del csv
Cisneros Gustavo

In [2]:
!pip install pandas

  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB 262.6 kB/s eta 0:00:41
   ---------------------------------------- 0.1/10.6 MB 363.1 kB/s eta 0:00:30
   ---------------------------------------- 0.1/10.6 MB 544.7 kB/s eta 0:00:20
    --------------------------------------- 0.2/10.6 MB 787.7 kB/s eta 0:00:14
   - -------------------------------------- 0.3/10.6 MB 1.0 MB/s eta 0:00:10
   - -------------------------------------- 0.5/10.6 MB 1.4 MB/s eta 0:00:08
   -- ------------------------------------- 0.6/10.6 MB 1.5 MB/s eta 0:00:07
   -- ------------------------------------- 0.8/10.6 MB 1.8 MB/s eta 0:00:06
   ---- ----------------------------------- 1.3/10.6 MB 2.6 MB/s eta 0:00:04
   ------ --------------------

In [7]:
import pandas as pd

data = pd.read_csv('salida.csv')
data

,NombreHotel,PrecioHotel,Dias
0,1. Blue Bay Hotel,113 €,30
1,2. Torre Marbella Salinas,108 €,30
2,3. Hotel Colón Salinas,124 €,30
3,4. Hostería Ecológica El Faro,184 €,30
4,5. Playa Canela Hotel Boutique,181 €,30
...,...,...,...
66,Big Ralph's Hostal,87 €,3
67,Patrocinado\nHotel Caridi,319 €,3
68,La Minga Hostal,104 €,3
69,Chaqana Lodge,83 €,3


In [ ]:
#Importaciones
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time


import scrapy

class QuotesSpiderSpider(scrapy.Spider):
    name = 'quotes_spider'
    allowed_domains = ['tripadvisor.es']
    start_urls = ['https://www.tripadvisor.es/Hotels-g297539-Salinas_Santa_Elena_Province-Hotels.html']
    visited_hotels = set()

    custom_settings = {
        'DOWNLOADER_MIDDLEWARES': {
            'tutorial.middlewares.CustomHeadersMiddleware': 543,
        }
    }
    
    def parse(self, response):
        # Selenium!
        # Opciones de navegación
        options = webdriver.ChromeOptions()
        options.add_argument('--start-maximized')
        options.add_argument('--disable-extensions')

        # Ruta al archivo ejecutable del ChromeDriver
        driver_path = r'C:\Users\Cisneros\tutorial\chrome drive\chromedriver-win64\chromedriver.exe'

        # Crear un objeto Service
        service = Service(driver_path)

        # Inicializar el controlador de Chrome
        driver = webdriver.Chrome(service=service, options=options)

        # Inicializar el Chrome con el link
        driver.get('https://www.tripadvisor.es/Hotels-g297539-Salinas_Santa_Elena_Province-Hotels.html')

        #Damos Click en el boton para entrada del dia
        WebDriverWait(driver, 5)\
        .until(EC.element_to_be_clickable((By.CSS_SELECTOR,'button.bqhCp _T u z Gz Wh sglCU'.replace(' ', '.'))))\
        .click()
        
        #Selecionamos el dia "1" de enero del 2024
        WebDriverWait(driver, 5)\
        .until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[3]/div[2]/div/div[2]/div/div[2]/div/div/div[1]/div/div[2]/div[2]/div[2]/div[1]/div[1]'.replace(' ', '.'))))\
        .click()

        #Selecionamos el dia "31" de enero del 2024
        WebDriverWait(driver, 5)\
        .until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[3]/div[2]/div/div[2]/div/div[2]/div/div/div[1]/div/div[2]/div/div[2]/div[5]/div[3]'.replace(' ', '.'))))\
        .click()

        #Cerramos la pestaña de actualizar
        WebDriverWait(driver, 10)\
        .until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[3]/div[2]/div/button'.replace(' ', '.'))))\
        .click()  

        #Esperamos que carge el chrome
        time.sleep(15)

        #Extraemos lo informacion de los hoteles
        listanombres = driver.find_elements(By.CSS_SELECTOR, '.jsTLT.K')
        listaprecios = driver.find_elements(By.CSS_SELECTOR, '.fwoto.f')

        for i, (nombre, precio) in enumerate(zip(listanombres, listaprecios)):
            nombreHotel = nombre.text
            precioHotel = precio.text
            #Use esta linea para comprobar la salida de datos
            print(nombreHotel + " " + precioHotel )
            if precioHotel !=None:
                #guardamos el el archivo csv
                yield {'NombreHotel': nombreHotel, 'PrecioHotel': precioHotel, 'Dias':"30"}


        #volvemos a inicializar el navegador para tener datos de los hoteles en servicio de 15 dias
        driver.get('https://www.tripadvisor.es/Hotels-g297539-Salinas_Santa_Elena_Province-Hotels.html')

        #Damos Click en el boton para entrada del dia
        WebDriverWait(driver, 25)\
        .until(EC.element_to_be_clickable((By.CSS_SELECTOR,'button.bqhCp _T u z Gz Wh sglCU'.replace(' ', '.'))))\
        .click()

        #Selecionamos el dia "1" de enero del 2024
        WebDriverWait(driver, 25)\
        .until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[3]/div[2]/div/div[2]/div/div[2]/div/div/div[1]/div/div[2]/div/div[2]/div[1]/div[1]'.replace(' ', '.'))))\
        .click()

        #Seleccionamos el dia 15 de enero del 2024
        WebDriverWait(driver, 25)\
            .until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[3]/div[2]/div/div[2]/div/div[2]/div/div/div[1]/div/div[2]/div/div[2]/div[3]/div[2]'.replace(' ', '.'))))\
            .click() 

        time.sleep(15)

        listanombres15 = driver.find_elements(By.CSS_SELECTOR, '.jsTLT.K')
        listaprecios15 = driver.find_elements(By.CSS_SELECTOR, '.fwoto.f')

        for i, (nombre15, precio15) in enumerate(zip(listanombres15, listaprecios15)):
            nombreHotel15 = nombre15.text
            precioHotel15 = precio15.text
            print(nombreHotel15 + " " + precioHotel15 )
            if precioHotel15 !=None:
                yield {'NombreHotel': nombreHotel15, 'PrecioHotel': precioHotel15, 'Dias':"15"}
        
        
        #volvemos a hacer el mismo proceso para los 3 dias de servicio de hoteles
        time.sleep(25)

        driver.get('https://www.tripadvisor.es/Hotels-g297539-Salinas_Santa_Elena_Province-Hotels.html')
        
        WebDriverWait(driver, 25)\
        .until(EC.element_to_be_clickable((By.CSS_SELECTOR,'button.bqhCp _T u z Gz Wh sglCU'.replace(' ', '.'))))\
        .click()
        
        WebDriverWait(driver, 25)\
        .until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[3]/div[2]/div/div[2]/div/div[2]/div/div/div[1]/div/div[2]/div/div[2]/div[1]/div[1]'.replace(' ', '.'))))\
        .click()
        WebDriverWait(driver, 25)\
            .until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[3]/div[2]/div/div[2]/div/div[2]/div/div/div[1]/div/div[2]/div[1]/div[2]/div[1]/div[3]'.replace(' ', '.'))))\
            .click() 

        time.sleep(15)
        listanombres1 = driver.find_elements(By.CSS_SELECTOR, '.jsTLT.K')
        listaprecios1 = driver.find_elements(By.CSS_SELECTOR, '.fwoto.f')

        for i, (nombre1, precio1) in enumerate(zip(listanombres1, listaprecios1)):
            nombre1 = nombre1.text
            precio1 = precio1.text
            print(nombre1 + " " + precio1 )
            if precio1 !=None:
                yield {'NombreHotel': nombre1, 'PrecioHotel': precio1, 'Dias':"3"}
        
        time.sleep(25)
        